In [60]:
from stsci.tools import capable
capable.OF_GRAPHICS = False

from pyraf import iraf
from pyraf.iraf import gemini, gemtools, gmos, onedspec
import fileSelect as fs
import copy

In [17]:
pwd

u'/home/mmarcano/Documents/GeminiReduceTTU/ExampleFolder'

First specified the directory where all the data was downloaded and extracted. Then defined the sqlite3 database. This contains the information of all the files and 

In [18]:
datadirall = '../rawExample/'
databasename = datadirall+'obsLog.sqlite3'
dbFile= datadirall+'obsLog.sqlite3'

 From the work_directory:
 Create the query dictionary of essential parameter=value pairs.

In [19]:
qd = {'Full':{'use_me':1,
       'Instrument':'GMOS-S','CcdBin':'2 4','RoI':'Full',
       'Disperser':'B600+_%','CentWave':485.0,'AperMask':'1.0arcsec',
       'Object':'AM2306-72%',
       'DateObs':'2007-06-05:2007-07-07'}
      }
# Make copy for the CenterSpec RoI:
qd['CenSp'] = copy.deepcopy(qd['Full'])
qd['CenSp'].update({'RoI':'CentSp','Object':'LTT9239'})

 In the tutorial there are two regions but here only one and it is full. Just changed the regions list to only Full. It should workl

In [20]:
gemtools.gemextn.unlearn()    # Disarm a bug in gbias
gmos.gbias.unlearn()
biasFlags = {
    'logfile':'biasLog.txt','rawpath':'../rawExample/','fl_vardq':'yes','verbose':'no'
}
regions = ['Full','CenSp']
for r in regions:
    # The following SQL generates the list of full-frame files to process.
    SQL = fs.createQuery('bias', qd[r])
    biasFiles = fs.fileListQuery(dbFile, SQL, qd[r])

    # The str.join() funciton is needed to transform a python list into a
    # comma-separated string of file names that IRAF can understand.
    if len(biasFiles) > 1:
        gmos.gbias(','.join(str(x) for x in biasFiles), 'MCbias'+r,
                   **biasFlags)

# Flat Field

In [22]:
# Set the task parameters.
qd['Full'].update({'DateObs':'*'})
qd['CenSp'].update({'DateObs':'*'})
gmos.gireduce.unlearn()
gmos.gsflat.unlearn()
# The response fitting should be done interactively.
flatFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_dark':'no',
    'fl_fixpix':'no','fl_oversize':'no','fl_vardq':'yes','fl_fulldq':'yes',
    'rawpath':'../rawExample/','fl_inter':'no','fl_detec':'yes',
    'function':'spline3','order':'13,11,28',
    'logfile':'gsflatLog.txt','verbose':'no'
    }
for r in regions:
    qr = qd[r]
    flatFiles = fs.fileListQuery(dbFile, fs.createQuery('gcalFlat', qr), qr)
    if len(flatFiles) > 0:
        gmos.gsflat (','.join(str(x) for x in flatFiles), 'MCflat'+r,
                bias='MCbias'+r, **flatFlags)


GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    

# Basic Reduction

In [24]:
# Set task parameters.
gmos.gsreduce.unlearn()
sciFlags = {
    'fl_over':'yes','fl_trim':'yes','fl_bias':'yes','fl_gscrrej':'no',
    'fl_dark':'no','fl_flat':'yes','fl_gmosaic':'yes','fl_fixpix':'no',
    'fl_gsappwave':'yes','fl_oversize':'no',
    'fl_vardq':'yes','fl_fulldq':'yes','rawpath':'../rawExample/',
    'fl_inter':'no','logfile':'gsreduceLog.txt','verbose':'no'
}
arcFlags = copy.deepcopy(sciFlags)
arcFlags.update({'fl_flat':'no','fl_vardq':'no','fl_fulldq':'no'})
stdFlags = copy.deepcopy(sciFlags)
stdFlags.update({'fl_fixpix':'yes','fl_vardq':'no','fl_fulldq':'no'})

# Arc exposures
for r in regions:
    qr = qd[r]
    arcFiles = fs.fileListQuery(dbFile, fs.createQuery('arc', qr), qr)
    if len(arcFiles) > 0:
        gmos.gsreduce (','.join(str(x) for x in arcFiles), bias='MCbias'+r,
                  **arcFlags)

# Std star exposures
r = 'CenSp'
stdFiles = fs.fileListQuery(dbFile, fs.createQuery('std', qd[r]), qd[r])
if len(stdFiles) > 0:
    gmos.gsreduce (','.join(str(x) for x in stdFiles), bias='MCbias'+r,
              flatim='MCflat'+r, **stdFlags)

# Science exposures
r = 'Full'
sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qd[r]), qd[r])
if len(sciFiles) > 0:
    gmos.gsreduce (','.join(str(x) for x in sciFiles), bias='MCbias'+r,
              flatim='MCflat'+r, **sciFlags)

GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                    Only saturated pixels will be flagged
GPREPARE: Using MDF defined in the header 1.0arcsec
WARNING - GIREDUCE: Bad Pixel Mask filename is an empty string
                   

# Wavelength Calibration (Check this later)

Image rectification and wavelength linearization depend upon the wavelength calibration, using the arc lamp exposures taken immediately before each sequence of science and standard star exposures (see Wavelength Calibration). In this case, the default medium-resolution line list will work well. The fit to the dispersion relation should be performed interactively, but for expediency we will use a previously determined functional fit. 


I had tpo manually put the name of the two arc Files. I am not sure how may to use. 

In [52]:
# Set task parameters
gmos.gswavelength.unlearn()
waveFlags = {
    'coordlist':'gmos$data/CuAr_GMOS.dat','fwidth':6,'nsum':50,
    'function':'chebyshev','order':5,
    'fl_inter':'no','logfile':'gswaveLog.txt','verbose':'no'
    }
# Must select specific wavecals to match science exposures.
prefix = 'gsS20070623S0'
for arc in ['071', '081', '091', '109']:
     gmos.gswavelength (prefix+arc, **waveFlags)

# Advanced Processing

The targets in this program were observed in 3 slit orientations, and a few exposures were obtained at each position. This provides an opportunity to combine the sequential exposures at each position to remove cosmic rays, rather than rejecting CRs on single frames using the gsreduce.fl_gscrrej+ flag or running the gemcrspec task. The combined exposures for each target are then wavelength calibrated, and sky subtracted. First set the processing parameters.

In [44]:
# Set task parameters.
gemtools.gemcombine.unlearn()
sciCombFlags = {
    'combine':'average','reject':'ccdclip',
    'fl_vardq':'yes','fl_dqprop':'yes',
    'logfile':'gemcombineLog.txt.txt','verbose':'no'
}
stdCombFlags = copy.deepcopy(sciCombFlags)
stdCombFlags.update({'fl_vardq':'no','fl_dqprop':'no'})
gmos.gstransform.unlearn()
transFlags = {
    'fl_vardq':'yes','interptype':'linear','fl_flux':'yes',
    'logfile':'gstransLog.txt'
}
# The sky regions should be selected with care, using e.g. prows/pcols:
#   pcols ("tAM2306b.fits[SCI]", 1100, 2040, wy1=40, wy2=320)
gmos.gsskysub.unlearn()
skyFlags = {
    'fl_oversize':'no','fl_vardq':'yes','logfile':'gsskysubLog.txt'
}

# Science Targets

the prefix was tto liong and could work with the scifiles and gemcombe. the function `','.join(prefix+str(x) for x in sciFiles)`

gsS20070623S0S20070623S0072

In [56]:
prefix='gs'

In [58]:
sciTargets = {
    'AM2306-721_a':{'arc':'gsS20070623S0071','sky':'520:720'},
    'AM2306-72_b':{'arc':'gsS20070623S0081','sky':'670:760,920:1020'},
    'AM2306-721_c':{'arc':'gsS20070623S0091','sky':'170:380,920:1080'}
}
for targ,p in sciTargets.iteritems():
    qs = qd['Full']
    qs['Object'] = targ
    # Fix up the target name for the output file
    sciOut = targ.split('-')[0]+targ[-1]
    sciFiles = fs.fileListQuery(dbFile, fs.createQuery('sciSpec', qs), qs)
    gemtools.gemcombine (','.join(prefix+str(x) for x in sciFiles),
                         sciOut, **sciCombFlags)
    gmos.gstransform (sciOut, wavtraname=p['arc'], **transFlags)
    gmos.gsskysub ('t'+sciOut, long_sample=p['sky'], **skyFlags)

WARNING - GEMCOMBINE: five or less images to combine.
-------------------------------------------------------------------------------
GSTRANSFORM -- Thu Nov  1 13:22:15 CDT 2018
 

inimages   = AM2306c
outimages  = 
outprefix  = t
fl_stran   = no
fl_wavtran = yes
wavtraname = gsS20070623S0091
database   = database
fl_vardq   = yes
interptype = linear
lambda1    = INDEF
lambda2    = INDEF
dx         = INDEF
nx         = INDEF
lambdalog  = no
ylog       = no
fl_flux    = yes
gratingdb  = gmos$data/GMOSgratings.dat
filterdb   = gmos$data/GMOSfilters.dat
key_dispaxis = DISPAXIS
dispaxis   = 1
sci_ext    = SCI
var_ext = VAR
dq_ext  = DQ

Transforming AM2306c.fits[SCI,1]
MDF row: 1

NOAO/IRAF V2.16 mmarcano@tux Thu 13:22:16 01-Nov-2018
  Transform AM2306c.fits[SCI,1] to tmpsci26692_2263.
  Conserve flux per pixel.
  User coordinate transformations:
    gsS20070623S0091_001
  Interpolation is linear.
  Using edge extension for out of bounds pixel values.
  Output coordinate parameters are:
  

In [68]:
onedspec.nsum=4
onedspec.sarith('stAM2306b.fits[SCI]', 'copy', '', 'estAM2306b.ms',
                  apertures='222-346x4')

In [92]:
len(biasFiles)

65